Basic simulation for the Posisorter system:
-

This code is made to run and simulate the posisorter system from Vanderlande. This model can run the provided Excel sheet with data from a real life posisorter. In this model some assumptions are made: 

In [2]:
import pandas as pd
from datetime import timedelta
import random


class Parcel:
    def __init__(self, parcel_id, arrival_time, length, width, height, weight, feasible_outfeeds):
        self.id = parcel_id
        self.arrival_time = arrival_time
        self.length = length
        self.width = width
        self.height = height
        self.weight = weight
        self.feasible_outfeeds = feasible_outfeeds
        self.sorted = False
        self.recirculated = False
        self.orientation = None  # 'parallel' or 'sequential'
        
    
    def effective_length(self) -> float:
        """
        Takes the effective length based on the sorting method.
        :return: Effective length based on the sorting method.
        """
        
        if self.orientation == 'sequential':
            return self.width  # Parcel is turned 
        return self.length  # Parcel is not turned
    
            
    def get_volume(self):
        return self.length * self.width * self.height
    
    
def compute_outfeed_time(parcel):
    base_time = 4.5

    # Volume classes
    volume = parcel.get_volume()
    if volume < 0.035:
        volume_class_delay = 0
    elif volume < 0.055:
        volume_class_delay = 1
    else:
        volume_class_delay = 2

    # Weight classes
    weight = parcel.weight
    if weight < 1700:
        weight_class_delay = 0
    elif weight < 2800:
        weight_class_delay = 1
    else:
        weight_class_delay = 2

    return base_time + volume_class_delay + weight_class_delay


class Outfeed:
    
    
    def __init__(self, max_length=3.0):
        self.max_length = max_length
        self.current_parcels = []
        self.current_length = 0.0
        self.time_until_next_discharge = 0.0 
        

    def can_accept(self, parcel):
        return self.current_length + parcel.effective_length() <= self.max_length
    

    def add_parcel(self, parcel):
        self.current_parcels.append((parcel, compute_outfeed_time(parcel)))
        self.current_length += parcel.effective_length()
        if len(self.current_parcels) == 1:
            #  Timer for the current parcel 
            self.time_until_next_discharge = self.current_parcels[0][1] 


    def update(self, time_step):
        if self.current_parcels:
            self.time_until_next_discharge -= time_step
            if self.time_until_next_discharge <= 0:
                parcel, _ = self.current_parcels.pop(0)
                self.current_length -= parcel.effective_length()
                print(f"Parcel {parcel.id} removed from outfeed")
                if self.current_parcels:
                    #  Timer for the next parcel in line
                    self.time_until_next_discharge = self.current_parcels[0][1]
                    
            
class PosiSorterSystem:
    
    
    def __init__(self, layout_df, sorting_method = 'random'):
        self.belt_speed = layout_df.loc[layout_df['Layout property'] == 'Belt Speed', 'Value'].values[0]
        self.num_outfeeds = 3  # Given in Excel sheet
        self.outfeeds = [Outfeed(max_length = 3.0) for _ in range(self.num_outfeeds)]
        self.recirculation_belt = []
        self.processed_parcels = []
        self.sorting_method = sorting_method  # 'parallel' or 'sequential'
        

    def sort_parcel(self, parcel):
        
        # Assign orientation based on selected sorting method
        if self.sorting_method == 'parallel':
            parcel.orientation = 'parallel'
        else: 
            parcel.orientation = 'sequential'
            
        for outfeed_index in parcel.feasible_outfeeds:
            outfeed = self.outfeeds[outfeed_index]
            if outfeed.can_accept(parcel):
                outfeed.add_parcel(parcel)
                parcel.sorted = True
                print(f"Parcel {parcel.id} sorted to outfeed {outfeed_index}")
                return
            
        parcel.recirculated = True
        self.recirculation_belt.append(parcel)
        
        print(f"Parcel {parcel.id} could not be sorted and was recirculated.")
        

    def remove_from_outfeeds(self, time_step):
        for outfeed in self.outfeeds:
            outfeed.update(time_step)
            

    def run_simulation(self, parcels):
        current_time = parcels[0].arrival_time
        end_time = parcels[-1].arrival_time + timedelta(seconds = 30)  # 30 seconds buffer to make sure all packages are sorted
        parcel_index = 0
        time_step = 0.1  # simulation time step in seconds

        while current_time <= end_time:
            print(f"\n--- Time: {current_time.time()} ---")

            # Check for parcel arrivals
            while parcel_index < len(parcels) and parcels[parcel_index].arrival_time <= current_time:
                parcel = parcels[parcel_index]
                parcel_index += 1
                self.processed_parcels.append(parcel)
                print(f"Parcel {parcel.id} arrived – feasible outfeeds: {parcel.feasible_outfeeds}")
                self.sort_parcel(parcel)

            self.remove_from_outfeeds(time_step)
            current_time += timedelta(seconds=time_step)

        self.print_summary()
        

    def print_summary(self):
        total = len(self.processed_parcels)
        sorted_count = sum(p.sorted for p in self.processed_parcels)
        recirculated_count = sum(p.recirculated for p in self.processed_parcels)

        print("\n--- Simulation Summary ---")
        print(f"Total parcels processed: {total}")
        print(f"Parcels sorted: {sorted_count}")
        print(f"Parcels recirculated: {recirculated_count}")
        print(f"Sorting success rate: {sorted_count / total:.2%}")

        if hasattr(self, 'drop_info'):
           print("\n--- Removed Parcel Summary ---")
           print(f"Initial amount of parcels: {self.drop_info['initial']}")
           print(f"Removed due to NaNs: {self.drop_info['na_dropped']}")
           print(f"Removed due to extreme outliers (data entry errors): {self.drop_info['outliers_dropped']}")
           print(f"Removed due to no feasible outfeeds: {self.drop_info['no_outfeeds_dropped']}")
           print(f"Removed due to non-chronological arrival times (scanning error): {self.drop_info.get('non_chrono_dropped', 0)}")
           print(f"Total parcels removed before processing: {self.drop_info['total_dropped']}")
            
def drop_non_chronological_arrivals(df):
    valid_times = []
    last_time = pd.Timestamp.min

    for i, time in enumerate(df["Arrival Time"]):
        if time >= last_time:
            valid_times.append(True)
            last_time = time
        else:
            valid_times.append(False)

    return df[valid_times].reset_index(drop=True)

def remove_outliers_iqr(df, columns):
    for col in columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 7 * IQR
        upper_bound = Q3 + 7 * IQR
        df = df[df[col].between(lower_bound, upper_bound)]
    return df


def drop_rows_without_true_outfeed(df, prefix="Outfeed"):
    outfeed_cols = [col for col in df.columns if col.startswith(prefix)]
    if not outfeed_cols:
        return df
    mask = df[outfeed_cols].any(axis=1)
    return df[mask]


def clean_parcel_data(parcels_df):
    drop_info = {}
    drop_info['initial'] = initial_count = len(parcels_df)

    parcels_df = parcels_df.dropna().reset_index(drop=True)
    after_na = len(parcels_df)
    drop_info['na_dropped'] = initial_count - after_na

    before_outliers = len(parcels_df)
    parcels_df = remove_outliers_iqr(parcels_df, ["Length", "Width", "Height"])
    after_outliers = len(parcels_df)
    drop_info['outliers_dropped'] = before_outliers - after_outliers

    before_outfeeds = len(parcels_df)
    parcels_df = drop_rows_without_true_outfeed(parcels_df)
    after_outfeeds = len(parcels_df)
    drop_info['no_outfeeds_dropped'] = before_outfeeds - after_outfeeds

    before_chrono = len(parcels_df)
    parcels_df = drop_non_chronological_arrivals(parcels_df)
    after_chrono = len(parcels_df)
    drop_info['non_chrono_dropped'] = before_chrono - after_chrono

    drop_info['total_dropped'] = drop_info['na_dropped'] + drop_info['outliers_dropped'] + drop_info['no_outfeeds_dropped'] + drop_info['non_chrono_dropped']

    return parcels_df, drop_info


def load_parcels_from_clean_df(df):
    parcels = []
    for _, row in df.iterrows():
        parcel_id = int(row['Parcel Number'])
        arrival_time = pd.to_datetime(row['Arrival Time'])
        length = float(row['Length'])
        width = float(row['Width'])
        height = float(row['Height'])
        weight = float(row['Weight'])
        feasible_outfeeds = [i for i, flag in enumerate([row['Outfeed 1'], row['Outfeed 2'], row['Outfeed 3']]) if flag]
        parcels.append(Parcel(parcel_id, arrival_time, length, width, height, weight, feasible_outfeeds))
    return sorted(parcels, key=lambda p: p.arrival_time)


def main():
    xls = pd.ExcelFile("PosiSorterData1.xlsx")
    parcels_df = xls.parse('Parcels')
    layout_df = xls.parse('Layout')

    parcels_df, drop_info = clean_parcel_data(parcels_df)

    parcels = load_parcels_from_clean_df(parcels_df)
    
    # Choose sorting method: 'parallel' or 'sequential'
    sorting_method = 'parallel'
    
    system = PosiSorterSystem(layout_df)
    system.drop_info = drop_info
    system.run_simulation(parcels)

if __name__ == "__main__":
    main()
    




--- Time: 09:00:01.212000 ---
Parcel 1 arrived – feasible outfeeds: [0]
Parcel 1 sorted to outfeed 0

--- Time: 09:00:01.312000 ---

--- Time: 09:00:01.412000 ---

--- Time: 09:00:01.512000 ---

--- Time: 09:00:01.612000 ---

--- Time: 09:00:01.712000 ---

--- Time: 09:00:01.812000 ---

--- Time: 09:00:01.912000 ---

--- Time: 09:00:02.012000 ---

--- Time: 09:00:02.112000 ---

--- Time: 09:00:02.212000 ---

--- Time: 09:00:02.312000 ---

--- Time: 09:00:02.412000 ---

--- Time: 09:00:02.512000 ---

--- Time: 09:00:02.612000 ---

--- Time: 09:00:02.712000 ---

--- Time: 09:00:02.812000 ---

--- Time: 09:00:02.912000 ---

--- Time: 09:00:03.012000 ---

--- Time: 09:00:03.112000 ---
Parcel 2 arrived – feasible outfeeds: [0, 2]
Parcel 2 sorted to outfeed 0

--- Time: 09:00:03.212000 ---

--- Time: 09:00:03.312000 ---

--- Time: 09:00:03.412000 ---

--- Time: 09:00:03.512000 ---

--- Time: 09:00:03.612000 ---

--- Time: 09:00:03.712000 ---

--- Time: 09:00:03.812000 ---

--- Time: 09:00:0